## Test of difference schemes
This is a one-dimensional simulation of the advection equation using the **FTCS** scheme in the range $0 < x < 100$.

$$
\frac{\partial u}{\partial t} = c \frac{\partial u}{\partial x} 
$$

In [1]:
import numpy as np

In [2]:
# Grid
nx = 101
xl, xr = 0., 100.
nghost = 1

dx = (xr - xl) / (nx - 1)
nxt = nx + 2 * nghost

# Courant number
nu = 0.8

# Advection speed (constant)
c = 1.

# Scheme
# 'ftcs', 'lf', 'lw', 'lw-rm', 'lw-mc', 'upwind',
scheme = 'lw-rm'

# Output dir
out_dir = 'lax-wendroff'

# Constant timestep
dt = nu * dx / c

# End time
t_end = 30.

# I/O
n_output = 1

# Initialize
t = 0.
nstep = 0


In [3]:
# Initialize grid
x = np.linspace(xl-nghost*dx, xr+nghost*dx, nxt)

In [4]:
# Initial conditions (step function)
u = np.where(x<50., 1., 0.)

In [5]:
# Time integration loop
while t < t_end:
    
    # Set Boundary conditions
    u[:nghost] = u[nghost]
    u[-nghost:] = u[-nghost-1]
    
    # Create left, right, and inner arrays to construct central differences
    u_right = u[nghost+1:]
    u_left = u[:-nghost-1]
    u_inner = u[nghost:-nghost]
    
    # Calculate new solution. One timestep in selected scheme.
    if scheme == 'ftcs':
        u_new_inner = u_inner - 0.5 * nu * (u_right - u_left)
        
    elif scheme == 'lf':
        u_new_inner = 0.5 * (u_right + u_left) - 0.5 * nu * (u_right - u_left)
        
    elif scheme == 'upwind':
        u_new_inner = u_inner - nu * (u_inner - u_left)
        
    elif scheme == 'lw':
        u_new_inner = u_inner - 0.5 * nu * (u_right - u_left) + 0.5 * nu * nu * (u_right - 2 * u_inner + u_left)
    
    elif scheme == 'lw-rm':
        un_left = 0.5 * (u_inner + u_left) - 0.5 * nu * (u_inner - u_left)
        un_right = 0.5 * (u_right + u_inner) - 0.5 * nu * (u_right - u_inner)
        u_new_inner = u_inner - nu * (un_right - un_left)
        
    else:
        print('Error: Unknown scheme. Exiting.')
        exit()

    # Create a copy so that we can still use u_new_inner.
    u_inner = u_new_inner.copy()
    
    # Extend u to include ghost cells again
    u = np.concatenate(([0], u_inner, [0]))
    
    # Update time
    t = t + dt
    nstep = nstep + 1

    # I/O
    if nstep % n_output == 0:
        u_inner.tofile(f'{out_dir}/{scheme}-{int(nstep/n_output):0>4d}.dat')
        